In [3]:
import cv2
import numpy as np
import os
import random
from pytube import Playlist, YouTube
import matplotlib.pyplot as plt
import pickle

In [4]:
def extract_frames_from_video(video_path, size = 256):
    """
    Extracts frames from a video.

    :param video_path: Path to the video file.
    :return: List of frames as numpy arrays.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video {video_path}")

    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = cv2.resize(frame, (size, size))
        frames.append(frame / size)
    cap.release()
    return frames

def extract_frames_from_video_min(video_path, start_minute, end_minute, size=256):
    """
    Extracts frames from a specified range of minutes in a video.

    :param video_path: Path to the video file.
    :param start_minute: The starting minute from which to extract frames.
    :param end_minute: The ending minute until which to extract frames.
    :param size: The size to which each frame is resized.
    :return: List of frames as numpy arrays.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video {video_path}")

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Convert times to frame numbers
    start_frame = int(start_minute * 60 * fps)
    end_frame = int(end_minute * 60 * fps)

    frames = []
    current_frame = 0
    while True:
        ret, frame = cap.read()
        if not ret or current_frame > end_frame:
            break

        if current_frame >= start_frame:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame = cv2.resize(frame, (size, size))
            frames.append(frame / size)

        current_frame += 1

    cap.release()
    return frames

def get_clips(frames_list, sequence_size, stride=10):
    """
    Splits the list of frames into clips of a fixed size with a given stride.

    :param frames_list: List of frames.
    :param sequence_size: The number of frames in each clip.
    :param stride: The step size between the start of each clip.
    :return: List of clips.
    """
    clips = []
    for i in range(0, len(frames_list) - sequence_size + 1, stride):
        clip = np.array(frames_list[i:i + sequence_size])  # Creating a clip
        clip = np.expand_dims(clip, axis=-1)  # Adding the channel dimension
        clips.append(clip)
    return clips

def get_training_set_from_video(video_path, sequence_size=16):
    frames = extract_frames_from_video(video_path)
    return np.array(get_clips(frames, sequence_size))

def visualize_frames(training_data, num_sequences=1, num_frames=10):
    """
    Visualizes frames from the training data.

    :param training_data: The training data containing sequences of frames.
    :param num_sequences: Number of sequences to visualize.
    :param num_frames: Number of frames to visualize per sequence.
    """
    for sequence in range(min(num_sequences, len(training_data))):
        fig, axs = plt.subplots(1, num_frames, figsize=(20, 2))
        for frame in range(num_frames):
            axs[frame].imshow(training_data[sequence][frame, :, :, 0], cmap='gray')
            axs[frame].axis('off')
        plt.show()
        
def visualize_as_video(training_data, sequence_idx=0, delay=30):
    """
    Visualizes a sequence of frames as a video.

    :param training_data: The training data containing sequences of frames.
    :param sequence_idx: Index of the sequence to visualize.
    :param delay: Delay between frames in milliseconds.
    """
    sequence = training_data[sequence_idx]
    for frame in sequence:
        frame_display = frame[:, :, 0] 
        cv2.imshow('Frame', frame_display)
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()

In [5]:
video_path = '/Users/idotzhori/Desktop/Desktop - Ido’s Laptop/autoencoderlight/full_video_2.mp4'

In [6]:
frames = extract_frames_from_video_min(video_path, 39, 40)

In [7]:
len(frames)

1800

In [8]:
with open('full_frames_39_40.pickle', 'wb') as f:
    pickle.dump(frames, f)

In [ ]:
with open('full_frames.pickle', 'rb') as f:
    frames = pickle.load(f)

In [9]:
test_set = np.array(get_clips(frames, 16))

In [10]:
for i in range(len(test_set)):
    visualize_as_video(test_set, sequence_idx=i, delay=10)

In [ ]:
model = load_model('256x256_3_8.h5')

In [ ]:
predictions = model.predict(test_set[0:5])